In [68]:
%reset -f
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [69]:
CSV_file = "SDS00001.CSV"

df = pd.read_csv(CSV_file, skiprows=[1])
df.describe()

Rhi = 4.7e6*5
Rlo = 110e3
G = (Rhi+Rlo)/Rlo
G *= 1.14

# Assign decent names to the columns and compute derivatives
df["t"] = df.Source - df.Source.min()
if (CSV_file == "SDS00001.CSV"):
    df["u_in_gt"] = df.CH4
    df["u_out"] = df.CH1 * G
    del df["CH1"]
    del df["CH4"]
    begin_at_value = +1000
    end_at_value = -1000
elif (CSV_file == "SDS00002.CSV"):
    df["u_in_gt"] = df.CH4
    df["u_out"] = df.CH1 * G
    del df["CH1"]
    del df["CH4"]
    begin_at_value = +1000
    end_at_value = -20
elif (CSV_file == "SDS00003.CSV"):
    df["u_in_gt"] = df.CH1
    df["u_out"] = df.CH3 * G
    del df["CH1"]
    del df["CH3"]
    begin_at_value = +1000
    end_at_value = -20
# Remove the silly-named original columns
del df["Source"]


desired_sample_rate = 1e3
time_span = df.t.max() - df.t.min()
desired_num_samples = int(time_span * desired_sample_rate)
downsample_factor = round(len(df) / desired_num_samples)
print(f"{downsample_factor=}")
num_rows = (len(df) // downsample_factor) * downsample_factor
df = df.iloc[:num_rows]
reshaped_df = df.values.reshape(-1, downsample_factor, df.shape[1])
downsampled_data = reshaped_df.mean(axis=1)
dsdf = pd.DataFrame(downsampled_data, columns=df.columns)
dsdf["dt"] = dsdf.t.diff()
dsdf["du_out_dt"] = dsdf.u_out.diff() / dsdf.dt
dsdf["du_out_dt_lpf"] = dsdf.du_out_dt.rolling(5).mean()

# Trim the dataframe to only include the part where the cap is being charged
begin_at = dsdf[dsdf.du_out_dt_lpf > begin_at_value].index[0] - 1
dsdf     = dsdf.iloc[begin_at:]
end_at   = dsdf[dsdf.du_out_dt_lpf < end_at_value].index[0] - 1
dsdf     = dsdf.iloc[:end_at-begin_at]
dsdf

# Trim the time base.
dsdf["t"] = dsdf.t - dsdf.t.min()
dsdf.describe()

downsample_factor=50179


,t,u_in_gt,u_out,dt,du_out_dt,du_out_dt_lpf
count,81.000000,81.000000,81.000000,8.100000e+01,81.000000,81.000000
mean,0.040143,119.712823,117.511921,1.003580e-03,2777.577229,2789.170343
std,0.023611,72.174749,74.646813,2.161118e-17,1219.729481,917.476403
min,0.000000,8.183662,1.343700,1.003580e-03,-309.606393,545.397025
25%,0.020072,50.151258,45.425409,1.003580e-03,2059.766829,2207.981415
50%,0.040143,125.680863,123.665069,1.003580e-03,2872.362140,2786.302058
75%,0.060215,185.139281,186.612803,1.003580e-03,3800.015195,3396.652289
max,0.080286,224.369796,226.295938,1.003580e-03,5293.239250,4472.092350


In [70]:
R_eq = (Rhi * Rlo) / (Rhi + Rlo)
C_out = 1e-9
tau_init = C_out * R_eq
tau_init

0.0001094875052943668

In [71]:
def predict_u_in(u_out, du_out_dt, tau):
    return u_out + tau * du_out_dt

dsdf["u_in_pred"] = predict_u_in(dsdf.u_out, dsdf.du_out_dt_lpf, tau_init)

fig = go.Figure()
fig.update_layout(
    height=400,
    hovermode='x',
    yaxis2=dict(title='u_c [V]', overlaying='y', side='right'),
)
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_gt, name="u_in_gt"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_out, name="u_out"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred, name="u_in_pred"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.du_out_dt_lpf, name="du_out/dt", yaxis='y2'))

In [72]:
fig = go.Figure()
fig.update_layout(height=400, hovermode='x')
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred-dsdf.u_in_gt, name="u_err"))